In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
model = load_model('sentimentmodel2.h5')

W0805 19:09:42.697374   248 deprecation.py:506] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0805 19:09:42.738265   248 deprecation.py:506] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    


In [52]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing import sequence

max_sentence_length = model.layers[0].input_shape[1]
lemmatizer = WordNetLemmatizer()
def process_text(texts):
    sentences = []
    for s in texts:
        replaced = re.sub(r'[^a-zA-z0-9\s]','', s.lower())

        #tokenize words
        words = word_tokenize(replaced)

        #remove common words
        #filtered_sentence = [w for w in words if not w in stop_words]

        #lemmatize words
        #lexicon = filtered_sentence
        lexicon = [lemmatizer.lemmatize(i) for i in words]
        sentences.append(lexicon)
        
    t = tokenizer.texts_to_sequences(sentences)
    p =  sequence.pad_sequences(t, maxlen=max_sentence_length)
    return p

prediction = model.predict_classes(process_text(["I love this movie. the plot was great", "I love this movie. the plot was great"]))

print(prediction)

[4 4]


In [83]:
def process_prediction(prediction):
    if prediction <= 1:
        return "Very Negative"
    elif prediction <= 1.9:
        return "Somewhat Negative"
    elif 1.9 <= prediction <= 2.1:
        return "Neutral"
    elif prediction <= 3:
        return "Somewhat Positive"
    else:
        return "Very Positive"
    
process_prediction(prediction[0])
    

'Very Positive'

In [12]:
import tweepy

In [22]:
CONSUMER_KEY = 'xtN9VUIDAyS5eoLC0gg035jQV'
CONSUMER_SECRET = 'kXla9AOJebgLARddBBMMwsBcbewWoZIWApoYffUcSMBPVvlffK'
ACCESS_TOKEN = '1053917274-KJg5VGA0JUki68TrWTuT5f54t83AhGseXKML1Jg'
ACCESS_TOKEN_SECRET = 'ZJOhe3vk2x7145M641OIYUHHTEbi2LmMuIcLJNErH6VGD'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


In [72]:
def get_dataset(query):
    results = api.search(query, count=100, lang='en')
    texts = [tweet.text for tweet in results]
    print(len(texts))
    return texts


In [102]:
data = get_dataset("beyonce")
inputs = process_text(data)
prediction = model.predict_classes(inputs)


83


In [103]:
import numpy as np

average = np.sum(prediction) / len(prediction)
print("Average Sentiment is ", average )
print(process_prediction(float(average)))

Average Sentiment is  2.5542168674698793
Somewhat Positive


In [126]:
import requests
url = 'https://g-sentiment.herokuapp.com/'
params ={'query': 'Bernie Sanders'}
response = requests.get(url, params)
print(response)
x = response.json()
print(x)

<Response [503]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)